<a href="https://colab.research.google.com/github/ilsilfverskiold/smaller-models-docs/blob/main/computer-vision/cook/image-classification/ViT_Huggingface_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Image classification using ViT/Swin Transformer with Hugging Face Trainer**

---

The pre-trained model we'll fine-tune here is set for a ViT model - google/vit-base-patch16-224 - but should work well for any swin transformer model as well.

Batch size is 32, epoch is 5 make sure to change these values to your preferences.

**Make sure you change the dataset to what you need.** My dataset I've used has both a training and a validation set, so change the code accordingly if you don't have a validation set.

In [ ]:
!pip install -q transformers datasets accelerate

In [ ]:
dataset_url = "ilsilfverskiold/traffic-camera-norway-images" # public dataset
model_checkpoint = "google/vit-base-patch16-224" # decide on your pre-trained model - see the huggingface hub
new_model_name = 'traffic-image-classification'
learning_rate = 5e-5
weight_decay = 0.01
epochs = 5
batch_size= 32

Fetch dataset from huggingface or import one from somewhere else. Make sure it has been properly processed before though so the images are in PIL format.Look into the cook book for processing and pushing a custom image dataset if it's new to you.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(dataset_url) # to fetch a private dataset use token=your_token

dataset

Look into the labels and set them so we can prepare the pre-trained model with them for fine-tuning.


In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

Preprocess the dataset for fine-tuning with ViT/ConvNEXT/Swin Transformer we'll use an image prcoessor to normalize. The image processor ensures that every input image conforms to expectations (input image size and pixel value range).

In [ ]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor

The code below is defining a set of image transformations that are applied to the training data. These transformations prepare images for input into a neural network by normalizing them and augmenting the dataset to improve model robustness.

In [ ]:
from torchvision.transforms import (
    Compose,
    Resize,
    Normalize,
    CenterCrop,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

train_transform = Compose([
    Resize(256),
    CenterCrop(224),
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transform = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    normalize,
])

def apply_transform(examples, transform):
    examples['pixel_values'] = [transform(image.convert('RGB')) for image in examples['image']]
    return examples

def set_dataset_transform(dataset, transform):
    dataset.set_transform(lambda examples: apply_transform(examples, transform))

In [ ]:
set_dataset_transform(dataset['train'], train_transform)
set_dataset_transform(dataset['validation'], val_transform)

Check that we now have another field called pixel_values for each item below.

In [ ]:
dataset['train'][0]

We use the labels we set up earlier from the dataset when importing the pre-trained model below, we also tell it to ignore the pre-defined labels that it previously have been trained on.

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True,
)

Set up your training metrics below.

**Accuracy** indicates overall correctness, **precision** measures the reliability of positive predictions, **recall** assesses the model's ability to identify all positive samples, and **F1 score** balances precision and recall, crucial in cases of class imbalance.

To understand this, if precision is relatively high, suggesting that when the model predicts an instance as positive, it is likely to be correct. However, if the recall is somewhat lower, this indicates that the model misses a significant portion of actual positive cases.

To put it into perspective, for complex tasks, **especially those involving highly imbalanced datasets** or where distinguishing classes is inherently challenging, an F1 score around 0.75 - 0.80 can be considered quite ok.

You'll need at least accuracy here though if you are considering to remove some of the metrics.

In [ ]:
import numpy as np
from datasets import load_metric

accuracy_metric = load_metric("accuracy")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='macro')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')

    metrics = {
        "accuracy": accuracy['accuracy'],
        "precision": precision['precision'],
        "recall": recall['recall'],
        "f1": f1['f1']
    }
    return metrics

The purpose of the collate_fn function below is to control how a list of samples (gathered from the dataset) is merged into a single batch. This function is crucial for ensuring that batches are structured properly before being fed into a model during training or evaluation.

In [ ]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

Set up your training arguments for the Hugging Face trainer. Leave a lot of it as you can stumble onto errors if you don't. Nevertheless, you may want to play around with the learning rate, batch size and epochs used.

In [ ]:
args = TrainingArguments(
    f"{new_model_name}",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    warmup_ratio=0.1,
    logging_steps=10,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Train the model. Remember to pay attention to the training loss and validation loss, both should consistently go down and if validation keeps going up while training loss keeps going down you may be overfitting the model. Accuracy should obviously go up, and if you see very small marginal increases for every epoch then you might have reached the limit of what you can achieve.

Don't worry too much if it fluctuates a bit, and try the model afterwards to see how it does as well.

In [ ]:
train_results = trainer.train()

trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.628200,0.572489,0.764359,0.793318,0.591823,0.652544
1,0.448600,0.463022,0.801178,0.796402,0.682381,0.721348
2,0.328500,0.439386,0.829161,0.823206,0.736606,0.772126
4,0.181400,0.436493,0.821797,0.799302,0.736248,0.763145


***** train metrics *****
  epoch                    =       4.9215
  total_flos               = 2168315527GF
  train_loss               =       0.4179
  train_runtime            =   0:12:04.01
  train_samples_per_second =       42.147
  train_steps_per_second   =        0.325


In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =     4.9215
  eval_accuracy           =     0.8292
  eval_f1                 =     0.7721
  eval_loss               =     0.4394
  eval_precision          =     0.8232
  eval_recall             =     0.7366
  eval_runtime            = 0:00:09.02
  eval_samples_per_second =     75.256
  eval_steps_per_second   =      2.438


We'll now save the model so we can do some inference on it.

In [ ]:
trainer.save_model('new_model')

In [ ]:
from transformers import pipeline

pipe = pipeline('image-classification', model='new_model')

I'm importing images from my Google Drive to test against. This is completely optional.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image

image_path = '/content/drive/MyDrive/image_to_test.jpg' # path to your image

image = Image.open(image_path)

results = pipe(image)
results

(Optional) I will also run it against a few images in the validation set to see what the results are.

In [ ]:
from PIL import Image

for i in range(200):
    image_data = dataset['validation'][i]['image']
    label_index = dataset['validation'][i]['label']

    if not isinstance(image_data, Image.Image):
        image = Image.open(image_data)
    else:
        image = image_data

    results = pipe(image)

    print(f"Results for image {i+1}:")
    print(results)
    print("Actual label:", id2label[label_index])
    print("----------------------------------")

If you're satisfied, log into HuggingFace with a token that you can get via Settings in your Hugging Face account. Remember that it needs both read and write access. It will ask you for this token below.

In [ ]:
!huggingface-cli login

In [ ]:
trainer.push_to_hub(new_model_name) # set Private=True if you want the model as private